In [1]:
import time
notebookstart= time.time()

In [2]:
import torch

C:\Users\_v010ch_\Anaconda3\envs\uly22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from typing import List, Tuple, Optional

import pandas as pd
import numpy as np

from PIL import Image
from pillow_heif import register_heif_opener
register_heif_opener() # for using Image.open for .heic without changes

from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
#import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
%matplotlib inline

In [6]:
DIR_DATA = os.path.join(os.getcwd(), 'data')
DIR_SUBM = os.path.join(os.getcwd(), 'subm')
DIR_SUBM_TRAIN = os.path.join(os.getcwd(), 'subm', 'train')
DIR_DATA_TRAIN = os.path.join(DIR_DATA, 'train')
DIR_DATA_TEST  = os.path.join(DIR_DATA, 'test')

In [7]:
test_img_names  = set(os.listdir(DIR_DATA_TEST))
train_img_names = set(os.listdir(DIR_DATA_TRAIN))
len(test_img_names), len(train_img_names)

(521, 530)

In [8]:
train_labels_df = pd.read_csv(os.path.join(DIR_DATA, 'train.csv'), sep=';', index_col=None)

In [9]:
def get_car_center(inp_tensor: torch.Tensor) -> Tuple[int, int]:

    car_cntr = (int((inp_tensor[2].int().item() - inp_tensor[0].int().item())/2 + inp_tensor[0].int().item()),
                int((inp_tensor[3].int().item() - inp_tensor[1].int().item())/2 + inp_tensor[1].int().item())
        )
    
    return car_cntr

In [10]:
def get_center_dist(inp_center: Tuple[int, int], inp_point: Tuple[int, int]) -> float:
    
    return np.sqrt((inp_center[0] - inp_point[0])**2 + \
                   (inp_center[1] - inp_point[1])**2)

In [15]:
def determine_targ_car(inp_results, inp_img_cntr: Tuple[int, int]) -> int:
    
    min_dist = 1000000
    min_idx  = -1
    
    for el in range(inp_results.xyxy[0].shape[0]):
        # учитываем только машины
        if inp_results.xyxy[0][el][5].int().item() != 2:
            continue
            
        # минимальные габариты учитываемых машин
        # в противном случае иногда ближе к центру оказываются машины например 27х54
        h = inp_results.xyxy[0][el][3] - inp_results.xyxy[0][el][1]
        w = inp_results.xyxy[0][el][2] - inp_results.xyxy[0][el][0]
        if w < 200 or h < 200:
            continue
            
            
        car_cntr = get_car_center(inp_results.xyxy[0][el])
        cur_dist = get_center_dist(inp_img_cntr, car_cntr)
        if cur_dist < min_dist:
            min_dist = cur_dist
            min_idx = el

    return min_idx

In [16]:
def create_feeatures(inp_fnames: List[str], inp_dir: str, inp_model, use_centr: Optional[bool] = False):
    
    ret_data = []

    for img_name in tqdm(inp_fnames): 
        #if 'heic' in img_name:
        #    heif_file = pyheif.read(os.path.join(inp_dir, img_name))
        #   img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
        #else:
        #    img = Image.open(os.path.join(inp_dir, img_name))
        img = Image.open(os.path.join(inp_dir, img_name))
        
        
        img = np.array(img)
        #results = model(np.array(img))
        results = model(img)
    
        if results.xyxy[0].shape != torch.Size([0, 6]):

            if use_centr:
                #img_cntr = (int(img_.shape[1]/2), int(img_.shape[0]/2))
                img_cntr = (int(img.shape[1]/2), int(img.shape[0]/2))
                target_goal = determine_targ_car(results, img_cntr)
            else:
                target_goal = 0

            if target_goal < 0:
                print(f'wtf2, {img_name}   {results.xyxy[0].shape}')
                continue
                
            h = results.xyxy[0][target_goal][3] - results.xyxy[0][target_goal][1]
            w = results.xyxy[0][target_goal][2] - results.xyxy[0][target_goal][0]
            results = results.xyxy[0][target_goal].numpy().tolist() + [h.item(), w.item()]
            
            # позволим алгоритмам самим выбирать как заполнить пропуски
            ret_data.append([img_name] + results)
            
            
        else:
            print(f'wtf, {img_name}   {results.xyxy[0].shape}')
            # позволим алгоритмам самим выбирать как заполнить пропуски
            #results = [0, 0, 0, 0, 0, 0, 0, 0]

# позволим алгоритмам самим выбирать как заполнить пропуски
#        ret_data.append([img_name] + results)
        
    ret_data = pd.DataFrame(ret_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class', 'h', 'w'])
        
    return ret_data

In [17]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')
model.classes = [0, 2]  # person and car
_ = model.cpu()

Using cache found in C:\Users\_v010ch_/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-3 Python-3.10.4 torch-1.12.0 CUDA:0 (GeForce GTX 1050 Ti, 4096MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


In [18]:
train_df = create_feeatures(train_img_names, DIR_DATA_TRAIN, model, use_centr = True) #use_centr
train_df = pd.merge(train_labels_df, train_df, how='left')
train_df.shape

 38%|██████████████████████████████▏                                                 | 200/530 [04:04<06:10,  1.12s/it]

wtf, img_1890.jpg   torch.Size([0, 6])


100%|████████████████████████████████████████████████████████████████████████████████| 530/530 [11:12<00:00,  1.27s/it]


(530, 10)

In [19]:
test_df = create_feeatures(test_img_names, DIR_DATA_TEST, model, use_centr = True) #use_centr
test_df.shape

  7%|█████▌                                                                           | 36/521 [00:44<10:14,  1.27s/it]

wtf2, img_1889.jpg   torch.Size([1, 6])


 11%|█████████                                                                        | 58/521 [01:10<08:52,  1.15s/it]

wtf, img_1888.jpg   torch.Size([0, 6])


 12%|█████████▍                                                                       | 61/521 [01:14<10:42,  1.40s/it]

wtf, img_2674.heic   torch.Size([0, 6])


 99%|███████████████████████████████████████████████████████████████████████████████▌| 518/521 [10:41<00:03,  1.14s/it]

wtf, img_2571.jpg   torch.Size([0, 6])


100%|████████████████████████████████████████████████████████████████████████████████| 521/521 [10:45<00:00,  1.24s/it]


(517, 9)

yolov5 не найдено машин:   
train: img_1890.jpg,    
test: img_1888.jpg, img_2674.heic, img_2571.jpg, img_1889.jpg(only person)   

In [20]:
sns.histplot(train_df, x='h')
plt.show()

C:\Users\_v010ch_\AppData\Local\Temp\ipykernel_20948\4050404586.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [21]:
sns.histplot(train_df, x='w')
plt.show()

C:\Users\_v010ch_\AppData\Local\Temp\ipykernel_20948\636209886.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [23]:
train_df['class'].value_counts()

2.0    529
Name: class, dtype: int64

In [24]:
test_df['class'].value_counts()

2.0    517
Name: class, dtype: int64

In [25]:
for el in ['x_min', 'y_min', 'x_max', 'y_max', 'h', 'w']:
    train_df[f'log_{el}'] = train_df[el].apply(lambda x: np.log(x))
    test_df[f'log_{el}'] = test_df[el].apply(lambda x: np.log(x))

In [28]:
train_df.head(20)

,image_name,distance,x_min,y_min,x_max,...,log_y_min,log_x_max,log_y_max,log_h,log_w
0,img_1596.jpg,4.88,1661.424072,1172.543701,2280.588135,...,7.066931,7.732189,7.448828,6.301344,6.428370
1,img_1600.jpg,1.54,1125.089722,1185.073486,3012.445312,...,7.077560,8.010507,7.907166,7.334073,7.542932
2,img_1601.jpg,3.68,1573.182739,1192.814087,2434.594727,...,7.084071,7.797536,7.610675,6.717596,6.758573
3,img_1603.jpg,2.22,2602.999512,1277.598389,3310.421875,...,7.152737,8.104831,7.522870,6.349610,6.561628
4,img_1605.jpg,3.73,1612.041016,1101.496704,2481.691650,...,7.004425,7.816696,7.562429,6.712978,6.768092
5,img_1606.jpg,4.52,1627.007202,1010.347107,2351.491943,...,6.918049,7.762805,7.447804,6.559251,6.585461
6,img_1607.jpg,5.40,1716.469482,1277.883301,2344.694580,...,7.152960,7.759910,7.541683,6.408721,6.442899
7,img_1608.jpg,6.21,1778.289307,1193.001953,2315.440186,...,7.084228,7.747355,7.449057,6.263854,6.286279
8,img_1612.jpg,7.12,2275.927734,1428.815552,2694.781738,...,7.264601,7.899072,7.457990,5.719799,6.037522
9,img_1613.jpg,5.26,1684.922485,1191.606812,2307.446289,...,7.083058,7.743897,7.474762,6.348046,6.433782


In [26]:
train_df.to_csv(os.path.join(DIR_DATA, 'train_upd.csv'), index = False)
test_df.to_csv(os.path.join(DIR_DATA, 'test_upd.csv'), index = False)